# 2. Finding an upper bound on CHSH, symmetrized version

We initialize the RepLAB library.

In [1]:
run ../../../replab_init

Adding RepLAB to the path
Adding RepLAB package to the path
Loading optim package for Octave...
Adding VPI to the path
Adding MOxUnit to the path
Adding embedded YALMIP to the path
Adding embedded SDPT3 solver to the path
Adding MOcov to the path


During the lecture, we found a form of the SDP that had only one optimization variable. We'll now see how to symmetrize the problem completely.

We first declare the one and only variable, before constructing the moment matrix.

In [2]:
y = sdpvar; % y is actually y_A0B0, but we save space by identifying y with it
C = [ 1  0  0  0  0
      0  1  0  0  0
      0  0  1  0  0
      0  0  0  1  0
      0  0  0  0  1];
A = [ 0  0  0  0  0
      0  0  0  1  1
      0  0  0  1 -1
      0  1  1  0  0
      0  1 -1  0  0];
X = C + A*y;
I_CHSH = 4*y; % it is <A0B0> + <A0B1> + <A1B0> - <A1B1>, and y_A1B1 = -y_A0B0.
optimize(X >= 0, -I_CHSH, sdpsettings('verbose', 0)); % sign change to maximize, and we don't show solver out
double(I_CHSH)

ans = NaN


To help with symmetrization, we'll use an equivalent form of the constraint X, and check the result.

Now, we want to find a change of basis that brings X into a block-diagonal form. This is easier to check on the basis matrices `C` and `A`, i.e. we want to find `U` such that `U*C*U'` and `U*A*U'` are block-diagonal. Let us use RepLAB for that.
First, remark that the moment matrix is invariant under the following signed permutations:

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, B_0, B_1, A_0, A_1 \right) $

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, -A_0, -A_1, -B_0, -B_1 \right) $

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, A_1, A_0, B_0, -B_1 \right) $

We now show how to use RepLAB to find the change of basis for the symmetry group that includes only the first symmetry.

In [3]:
g1 = [1 4 5 2 3]; % permutation of parties
g2 = [1 -2 -3 -4 -5]; % sign flip everywhere, note the signed permutation convention
g3 = [1 3 2 4 -5]; % additional symmetry

We build the symmetry group from those generators, which are signed permutations on five elements. For the permutation of parties only, the order (=size) of the group is 2. When adding the other symmetries, the order of the group should be 16.

In [4]:
nElements = 5;
G = replab.signed.Permutations(nElements).subgroup({g1 g2 g3});
G.order

ans =
    16


If necessary, we can also expand the group elements (use `G.elements.at(2)` to get the second element for example).

In [5]:
G.elements

ans =

Indexed family of 16 elements
 1 = [1, 2, 3, 4, 5]    
 2 = [1, -3, -2, -4, 5] 
 3 = [1, 2, -3, 5, 4]   
 4 = [1, 3, -2, -5, 4]  
 5 = [1, 4, 5, 2, 3]    
 6 = [1, -5, -4, -2, 3] 
 7 = [1, 4, -5, 3, 2]   
 8 = [1, 5, -4, -3, 2]  
 9 = [1, -4, 5, -3, -2] 
10 = [1, -5, 4, 3, -2]  
11 = [1, -4, -5, -2, -3]
12 = [1, 5, 4, 2, -3]   
13 = [1, -2, 3, -5, -4] 
14 = [1, -3, 2, 5, -4]  
15 = [1, -2, -3, -4, -5]
16 = [1, 3, 2, 4, -5]   



Now, the representation we need is composed of signed permutation matrices; as we are considering a group of signed permutations, this is the group *natural representation*. Representations in RepLAB are described using the images of the generators. By calling `G.sampleUniformly`, we get random elements from the group, and by calling `rep.image(g)` we get the image of a group element.

In [6]:
rep = G.naturalRep

rep =

Orthogonal representation by images
                 dimension: 5                                
                     field: 'R'                              
   frobeniusSchurIndicator: []                               
                     group: replab.signed.PermutationSubgroup
    inverseImages_internal: 1 x 3 cell                       
isDivisionAlgebraCanonical: []                               
             isIrreducible: []                               
                 isUnitary: true                             
          trivialDimension: []                               
        images_internal{1}: 5 x 5 double                     
        images_internal{2}: 5 x 5 double                     
        images_internal{3}: 5 x 5 double                     



In [7]:
g = G.sampleUniformly
full(rep.image(g)) % we use full, as signed permutation representations return sparse matrices

g =

   1  -2   3  -5  -4

ans =

   1   0   0   0   0
   0  -1   0   0   0
   0   0   1   0   0
   0   0   0   0  -1
   0   0   0  -1   0



Now, we decompose the representation into irreducible components. For that, we call `rep.decomposition`. In the answer I(m)xR(d), we express that the component has $m$ copies (multiplicity) of a `R`eal representation of dimension `d`. We then play with the indices of the component, and copy.

Hint: explore `rep.decomposition.component(i).irrep(j)` for $(i,j) = (1,1), (2,1), (3,1)$.

In [8]:
rep.decomposition

ans =

Orthogonal representation
                 dimension: 5                                             
                     field: 'R'                                           
   frobeniusSchurIndicator: []                                            
                     group: replab.signed.PermutationSubgroup             
isDivisionAlgebraCanonical: []                                            
             isIrreducible: []                                            
                 isUnitary: true                                          
                    parent: Orthogonal representation by images           
          trivialDimension: []                                            
               basis.(:,1): [1, -1.4207e-18, 0, 2.9938e-18, 4.0608e-19].' 
               basis.(:,2): [0, -0.31163, 0.63474, 0.22847, -0.66918].'   
               basis.(:,3): [0, 0.63474, 0.31163, 0.66918, 0.22847].'     
               basis.(:,4): [0, -0.59206, 0.38661, 0.14528, 0.69202

In [9]:
rep.decomposition.component(1).irrep(1)

ans =

Orthogonal trivial irreducible real-type subrepresentation
                 dimension: 1                                            
                     field: 'R'                                          
   frobeniusSchurIndicator: 1                                            
                     group: replab.signed.PermutationSubgroup            
isDivisionAlgebraCanonical: []                                           
             isIrreducible: true                                         
                 isUnitary: true                                         
                    parent: Orthogonal representation by images          
          trivialDimension: 1                                            
               basis.(:,1): [1, -1.4207e-18, 0, 2.9938e-18, 4.0608e-19].'



We now ask for the change of basis matrix, and verify that `A` and `C` block-diagonalize.

In [10]:
U = rep.decomposition.basis
U'*C*U
U'*A*U

U =

   1.00000   0.00000   0.00000   0.00000   0.00000
  -0.00000  -0.31163   0.63474  -0.59206  -0.38661
   0.00000   0.63474   0.31163   0.38661  -0.59206
   0.00000   0.22847   0.66918   0.14528   0.69202
   0.00000  -0.66918   0.22847   0.69202  -0.14528

ans =

   1.0000e+00   8.5497e-19   1.1944e-18   1.5571e-18   2.5620e-18
   8.5497e-19   1.0000e+00   3.7031e-18  -1.5730e-16  -1.2659e-16
   1.1944e-18   3.7031e-18   1.0000e+00  -8.8895e-18   1.2365e-17
   1.5571e-18  -1.5730e-16  -8.8895e-18   1.0000e+00  -8.6679e-18
   2.5620e-18  -1.2659e-16   1.2365e-17  -8.6679e-18   1.0000e+00

ans =

  -9.6602e-36   1.2091e-18   1.6891e-18  -2.2020e-18  -3.6232e-18
   1.2091e-18   1.4142e+00   5.0924e-17  -4.2516e-16  -3.7420e-16
   1.6891e-18   4.3752e-17   1.4142e+00  -2.8723e-17   9.5131e-17
  -2.2020e-18  -4.3356e-16  -1.3871e-16  -1.4142e+00   9.2615e-17
  -3.6232e-18  -4.1594e-16   1.2396e-16  -1.3807e-17  -1.4142e+00



Once we have found the change of basis matrix, we transform our moment matrix `X`, and naturally it should have a block diagonal structure following the one of `C` and `A` -- for that, it may be necessary to kill small off-block coefficients.

In [11]:
X = (U'*C*U) + (U'*A*U)*y; % equivalent to the constraint above
optimize(X >= 0, -I_CHSH); % sign change to maximize
double(I_CHSH)

ans = NaN


As an exercice, recover an algebraic basis from `U`, guess the form of the semidefinite program when fully block-diagonal (i.e. for the full group of CHSH symmetries).
**Remark that now the problem can be solved by hand to recover the** $2\sqrt{2}$ **bound, as the SDP matrix is fully diagonal, and thus corresponds to linear inequalities.**